In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import json
import datetime

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
dataset_name = "GEMM_EX_CNNLSTM_1"

In [4]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
if not os.path.exists("version"):
    os.makedirs("version")
version_dir = "version/" + timestamp 

timestamp

'20201204-231028'

vocabulary = np.genfromtxt("static/vocabulary_gpa.csv", delimiter="\n", dtype=np.int64)
vocab_size = vocabulary.shape[0]
vocab_size

In [5]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BUFFER_SIZE"] = 200000
param_list["VOCAB_SIZE"] = 33476    #vocab_size
param_list["SHUFFLE_SEED"] = 102
param_list["BUFFER_SIZE"] = 200000

In [26]:
Hparam_list = dict()

Hparam_list["BATCH_SIZE"] = 2   ###1024
Hparam_list["EPOCHS"] = 10000        # EarlyStopping
Hparam_list["ENCODER_1_NEURONS"] = 64
Hparam_list["ENCODER_1_DROPOUT"] = 0.1
Hparam_list["DECODER_1_NEURONS"] = 64
Hparam_list["DECODER_1_DROPOUT"] = 0.1
Hparam_list["FILTERS"] = 32
Hparam_list["KERNEL_SIZE"] = 3
Hparam_list["POOL_SIZE"] = 2

In [27]:
train_set = pd.read_csv("data/{}_train_set.csv".format(dataset_name), dtype=np.float32)

In [28]:
x_train = tf.data.Dataset.from_tensor_slices(train_set.values[:-param_list["FUTURE_TARGET"]]).window(param_list["PAST_HISTORY"], 1, 1, True)
# As dataset.window() returns "dataset", not "tensor", need to flat_map() it with sequence length
x_train = x_train.flat_map(lambda x: x.batch(param_list["PAST_HISTORY"])) 
x_train = x_train.map(lambda x: tf.reshape(x, [16, 2, 1]))
x_train = x_train.batch(Hparam_list["BATCH_SIZE"])

In [29]:
for x in x_train.take(1):
    print(x.shape)

(2, 16, 2, 1)


In [36]:
keras.layers.Conv1D(32, 4)(x)

InvalidArgumentError: Computed output size would be negative: -1 [input_size: 2, effective_filter_size: 4, stride: 1] [Op:Conv2D]

In [9]:
y_train = tf.data.Dataset.from_tensor_slices(train_set["gpa"].values[param_list["PAST_HISTORY"]:]).window(param_list["FUTURE_TARGET"], 1, 1, True)
y_train = y_train.flat_map(lambda y: y.batch(param_list["FUTURE_TARGET"]))
y_train = y_train.batch(Hparam_list["BATCH_SIZE"])

In [10]:
train_data = tf.data.Dataset.zip((x_train, y_train))#.shuffle(param_list["BUFFER_SIZE"], seed=param_list["SHUFFLE_SEED"])   # shuffle not working

In [11]:
val_set = pd.read_csv("data/{}_val_set.csv".format(dataset_name), dtype=np.float32)

In [12]:
x_val = tf.data.Dataset.from_tensor_slices(val_set.values[:-param_list["FUTURE_TARGET"]]).window(param_list["PAST_HISTORY"], 1, 1, True)
x_val = x_val.flat_map(lambda x: x.batch(param_list["PAST_HISTORY"]))
x_val = x_val.batch(Hparam_list["BATCH_SIZE"])

In [13]:
y_val = tf.data.Dataset.from_tensor_slices(val_set["gpa"].values[param_list["PAST_HISTORY"]:]).window(param_list["FUTURE_TARGET"], 1, 1, True)
y_val = y_val.flat_map(lambda y: y.batch(param_list["FUTURE_TARGET"]))
y_val = y_val.batch(Hparam_list["BATCH_SIZE"])

In [14]:
val_data = tf.data.Dataset.zip((x_val, y_val))

In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters=param_list["FILTERS"], kernel_size=param_list["KERNEL_SIZE"], padding="causal", activation='relu'))
model.add(keras.layers.MaxPooling1D(pool_size=param_list["POOL_SIZE"]))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(param_list["NUM_1_NEURONS"])))
model.add(keras.layers.Dropout(param_list["DROPOUT_2"]))
model.add(keras.layers.RepeatVector(param_list["FUTURE_TARGET"]))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(param_list["NUM_2_NEURONS"], return_sequences=True)))
model.add(keras.layers.Dropout(param_list["DROPOUT_2"]))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(param_list["VOCAB_SIZE"], activation='softmax')))

model.compile(optimizer=keras.optimizers.Nadam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
model_history = model.fit(train_data, epochs=Hparam_list["EPOCHS"], validation_data=val_data, callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)])

Epoch 1/10000
1550/1550 [==============================] - 556s 359ms/step - loss: 4.3839 - accuracy: 0.3805 - val_loss: 4.6168 - val_accuracy: 0.4032
Epoch 2/10000
1550/1550 [==============================] - 557s 360ms/step - loss: 3.7325 - accuracy: 0.4313 - val_loss: 4.5191 - val_accuracy: 0.4113
Epoch 3/10000
1550/1550 [==============================] - ETA: 0s - loss: 3.5353 - accuracy: 0.4352

KeyboardInterrupt: 

In [17]:
model.save(version_dir)

ValueError: Model <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001D9EF05C948> cannot be saved because the input shapes have not been set. Usually, input shapes are automatically determined from calling `.fit()` or `.predict()`. To manually set the shapes, call `model.build(input_shape)`.